In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Classification with Neural Networks

# Pipeline

1. **Import important libraries.**
2. **Get Dataset Ready.**
3. **Build Neural Network Classification Model.**
4. **Pick a Loss function and Optimizer.**
5. **Build a training loop.**
6. **Evaluate the model.**
7. **Optimize the model.**
8. **Save the model.**

## 1. Import important libraries.

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataloader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

ImportError: cannot import name 'Dataloader' from 'torch.utils.data' (/opt/conda/lib/python3.10/site-packages/torch/utils/data/__init__.py)